# Environment

In [1]:
import os
import io
import json
import requests
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from collections import defaultdict
from collections import namedtuple
from glob import glob

In [2]:
# Adjust CWD
directory = os.getcwd().replace('\\notebooks', '')
os.chdir(directory)
wd = os.getcwd()

# Baseline Results

In [3]:
# create model validation and test accuracy storage
va_dict = defaultdict(list)
ta_dict = defaultdict(list)

# iterate through model directories
for model_dir in sorted(glob(f'training_logs\\baseline/*')):
    
    # get model id based on model directory
    _, model_id = os.path.split(model_dir) #altered as original code included "training_output\" in model names
    
    # split model_id into model_name and training_iter
    model_name, _ = model_id.rsplit('_', 1)
    
    # read validation accuracy from training logs 
    train_log = pd.read_csv(f'{model_dir}/train_log.csv')
    va = train_log.valid_accuracy.tolist()
    
    # append validation and test accuracy to corresponding list
    # append logs if they're the first logs of the given model
    # or if they're of the same length as the previous logs
    # (avoid storing logs of a model that hasn't finished trainig yet) 
    if len(va_dict[model_name]) == 0 or len(va_dict[model_name][0]) == len(va):
        # read test accuracy from test logs
        with open(f'{model_dir}/test_logs.json') as f:
            ta = json.load(f)['accuracy']
            
        va_dict[model_name] += [va]
        ta_dict[model_name] += [ta]

# compute accuracy mean and SE for each model
Logs = namedtuple('Logs', ['va_mean', 'va_se', 'ta_mean', 'ta_se'])
logs = {}
for k, v in va_dict.items():
    # print number of training iters for each model
    print(f'{k}: {len(v)}')

    # calculate the mean and standard error of valid. accuracy
    va = np.array(v)
    # va = np.array([ma(x, 10) for x in va])
    va_mean = np.mean(va, 0)
    va_se = np.std(va, 0) / np.sqrt(va.shape[0])
    
    # calculate the mean and standard error of test accuracy
    ta = np.array(ta_dict[k])
    ta_mean = np.mean(ta)
    ta_se = np.std(ta) / np.sqrt(len(ta))
    
    # save validation and test logs
    logs[k] = Logs(va_mean, va_se, ta_mean, ta_se)

# Initialize lists to store data
model_names = []
validation_accuracy = []
testing_accuracy = []

# Iterate over the logs
for var, log in logs.items():
    # Extract mean and standard error for validation accuracy
    va_mean = log.va_mean[-1]
    va_se = log.va_se[-1]
    
    # Extract mean and standard error for testing accuracy
    ta_mean = log.ta_mean
    ta_se = log.ta_se
    
    # Append data to lists
    model_names.append(f'{var}')
    validation_accuracy.append(f'{100 * va_mean:.3f} ± {100 * va_se:.3f}')
    testing_accuracy.append(f'{100 * ta_mean:.6f} ± {100 * ta_se:.6f}')

# Create DataFrame
df = pd.DataFrame({
    'Model Name': model_names,
    'Validation Accuracy %': validation_accuracy,
    'Testing Accuracy %': testing_accuracy
})

# Display DataFrame
df

FasterRCNN_FPN_1100_square: 5
FasterRCNN_FPN_800_square: 5
RCNN_128_square: 5
RCNN_64_square: 5


,Model Name,Validation Accuracy %,Testing Accuracy %
0,FasterRCNN_FPN_1100_square,98.540 ± 0.115,98.523489 ± 0.113897
1,FasterRCNN_FPN_800_square,98.362 ± 0.067,98.308723 ± 0.083610
2,RCNN_128_square,98.382 ± 0.062,97.973155 ± 0.074494
3,RCNN_64_square,97.996 ± 0.080,97.731544 ± 0.125055


# Enhanced Results

## Choose Augmentation Method

In [4]:
logs_dir = 'training_logs\\random_se_b'
#logs_dir = 'training_logs\\proportional_se_b'

## Load Model Training Logs

In [5]:
# create model validation and test accuracy storage
va_dict = defaultdict(list)
ta_dict = defaultdict(list)

# iterate through model directories
for model_dir in sorted(glob(f'{logs_dir}/*')):
    
    # get model id based on model directory
    _, model_id = os.path.split(model_dir)
    
    # extract model_name and var from model_id
    model_name, var = model_id.rsplit('_', 1)
    
    # remove the "_square_X" portion from model_name
    model_name = model_name.split('_square')[0]
    
    # append validation and test accuracy to corresponding list
    # read validation accuracy from training logs 
    train_log = pd.read_csv(f'{model_dir}/train_log.csv')
    va = train_log.valid_accuracy.tolist()
    va_dict[f'{model_name}_{var}'].append(va)
    
    # read test accuracy from test logs
    with open(f'{model_dir}/test_logs.json') as f:
        ta = json.load(f)['accuracy']
    ta_dict[f'{model_name}_{var}'].append(ta)

# compute accuracy mean and SE for each model
Logs = namedtuple('Logs', ['va_mean', 'va_se', 'ta_mean', 'ta_se'])
logs = {}
for model_id, v in va_dict.items():
    # print number of training iters for each model
    print(f'{model_id}: {len(v)}')

    # calculate the mean and standard error of valid. accuracy
    va = np.array(v)
    va_mean = np.mean(va, axis=0)
    va_se = np.std(va, axis=0) / np.sqrt(va.shape[0])
    
    # calculate the mean and standard error of test accuracy
    ta = np.array(ta_dict[model_id])
    ta_mean = np.mean(ta)
    ta_se = np.std(ta) / np.sqrt(len(ta))
    
    # save validation and test logs
    logs[model_id] = Logs(va_mean, va_se, ta_mean, ta_se)

FasterRCNN_FPN_1100_var1.5: 5
FasterRCNN_FPN_1100_var1.75: 5
FasterRCNN_FPN_1100_var2.0: 5
FasterRCNN_FPN_800_var1.5: 5
FasterRCNN_FPN_800_var1.75: 5
FasterRCNN_FPN_800_var2.0: 5
RCNN_128_var1.5: 5
RCNN_128_var1.75: 5
RCNN_128_var2.0: 5
RCNN_64_var1.5: 5
RCNN_64_var1.75: 5
RCNN_64_var2.0: 5


## Display Results

In [6]:
# Initialize lists to store data
model_names = []
validation_accuracy = []
testing_accuracy = []

# Iterate over the logs
for var, log in logs.items():
    # Extract mean and standard error for validation accuracy
    va_mean = log.va_mean[-1]
    va_se = log.va_se[-1]
    
    # Extract mean and standard error for testing accuracy
    ta_mean = log.ta_mean
    ta_se = log.ta_se
    
    # Append data to lists
    model_names.append(f'{var}')
    validation_accuracy.append(f'{100 * va_mean:.3f} ± {100 * va_se:.3f}')
    testing_accuracy.append(f'{100 * ta_mean:.6f} ± {100 * ta_se:.6f}')

# Create DataFrame
df = pd.DataFrame({
    'Model Name': model_names,
    'Validation Accuracy %': validation_accuracy,
    'Testing Accuracy %': testing_accuracy
})

# Display DataFrame
df

,Model Name,Validation Accuracy %,Testing Accuracy %
0,FasterRCNN_FPN_1100_var1.5,98.646 ± 0.028,98.496644 ± 0.077341
1,FasterRCNN_FPN_1100_var1.75,98.824 ± 0.056,98.510067 ± 0.058200
2,FasterRCNN_FPN_1100_var2.0,98.738 ± 0.093,98.563758 ± 0.055668
3,FasterRCNN_FPN_800_var1.5,98.498 ± 0.038,98.268455 ± 0.051639
4,FasterRCNN_FPN_800_var1.75,98.518 ± 0.129,98.174497 ± 0.091826
5,FasterRCNN_FPN_800_var2.0,98.348 ± 0.105,98.268456 ± 0.074494
6,RCNN_128_var1.5,98.476 ± 0.099,98.040270 ± 0.039818
7,RCNN_128_var1.75,98.508 ± 0.066,98.174496 ± 0.058200
8,RCNN_128_var2.0,98.540 ± 0.077,98.067114 ± 0.087814
9,RCNN_64_var1.5,98.362 ± 0.051,98.214765 ± 0.055668
